# Road Following - Data Collection

If your motors and camera are working then you can start this activtiy.  This is where we get into the AI!  We are going to train an AI model to drive the car along a road!  This AI model is called an artifical neural network (ANN), because it tries to mimic the behaviour of the human brain (which is a real neural network!).  The technique for training the ANN is called deep learning.  It's a really important (and complex!) technique in AI.

## General Approach
The general approach is as follows.

**a. Data Collection**
1. Get the car to take a picture of the road ahead.
2. Tell the car which direction to go if it sees this road situation.
3. Go back to step 1 until you have enough images

**b. Train the Model**
1. Present the images to the deep learning neural network
2. The neural network will 'learn' what to do when it sees not just the road situations you have given it, but *any* road situation.

**c. Optimise the Model**
1. Optimise the model so it works well on the Jetbot

**d. Drive!**
1. Test your robot on the road.

This notebook covers step a - the data collection.



## Data Collection  

This is what we will do:

1. Place the JetBot in different positions on the road (offset from center, different angles, etc)
2. Display the live camera feed from the robot
3. Click on the image to place a 'green dot', which corresponds to the target direction we want the robot to travel, on the image.
4. Store the X, Y values of this green dot along with the image from the robot's camera

So how do you decide exactly where to place the target for this example?  Here is a guide we think may help

1.  Look at the live video feed from the camera
2.  Imagine the path that the robot should follow (try to approximate the distance it needs to avoid running off road etc.)
3.  Place the target as far along this path as it can go so that the robot could head straight to the target without 'running off' the road.


### Import Libraries

So lets get started by importing all the required libraries for "data collection" purpose. 

In [1]:
# IPython Libraries for display and widgets
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Basic Python packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### Data Collection

Now run the following code to display the camera image.  Click on the image, where you think the robot should go when it sees the road.  The image you recorded will appear on the right.

Record around 20 images, varying the road situation so the neural network will get lots of different situations to learn.


In [3]:
from jupyter_clickable_image_widget import ClickableImageWidget

DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')

camera = Camera()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
count_widget = ipywidgets.IntText(description='count')
# manually update counts at initialization
count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        #dataset.save_entry(category_widget.value, camera.value, x, y)
        uuid = 'xy_%03d_%03d_%s' % (x, y, uuid1())
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(camera_widget.value)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    count_widget
])

display(data_collection_widget)

Directories not created because they already exist


Again, let's close the camera conneciton properly so that we can use the camera in other notebooks.

In [5]:
camera.stop()

### Next

Once you've collected enough data, we'll need to copy that data your mentor's computer for training the neural network. First, we can call the following terminal command to compress our dataset folder into a single zip file.  


In [4]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}

You should see a file named road_following_<Date&Time>.zip in the Jupyter Lab file browser. You should download the zip file using the Jupyter Lab file browser by right clicking and selecting Download.  Put the file on a USB stick and give it to your mentor.